In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = '../data/cifar/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]


In [22]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.fc1 = nn.Linear(8*8*8,32)
        self.fc2 = nn.Linear(32,2)
        
    def forward(self,x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)),2)  # 2 是max_pool2d 的参数
        out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
        out = out.view(-1,8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        
        return out

In [23]:
img,_ = cifar2[0]

In [24]:
model = Net()
model(img.unsqueeze(0))

tensor([[0.1054, 0.1153]], grad_fn=<AddmmBackward>)

In [26]:
# loop
import datetime


def training_loop(n_epochs,optimizer,model,loss_fn,train_loader):
    for epoch in range(1,n_epochs+1):
        loss_train = 0.0
        for imgs,labels in train_loader:
            
            outputs = model(imgs)
            loss = loss_fn(outputs,labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_train += loss.item()
        if epoch==1 or epoch % 10 == 0:
            print('{} Epoch {},Training loss {}'.format(
                datetime.datetime.now(),epoch,loss_train/len(train_loader)))  # 计算每个batch的平均损失

In [27]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
model = Net()
optimizer = optim.SGD(model.parameters(),lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()


training_loop(n_epochs=100,
             optimizer=optimizer,
             model=model,
             loss_fn=loss_fn,
             train_loader=train_loader)

2020-07-19 17:14:25.352883 Epoch 1,Training loss 0.5701384637386177
2020-07-19 17:14:45.797915 Epoch 10,Training loss 0.33377454121401356
2020-07-19 17:15:08.995031 Epoch 20,Training loss 0.2964806552905186
2020-07-19 17:15:31.422398 Epoch 30,Training loss 0.2676278444801926
2020-07-19 17:15:53.826521 Epoch 40,Training loss 0.24656456239094401
2020-07-19 17:16:16.482019 Epoch 50,Training loss 0.22621194576951348
2020-07-19 17:16:44.447339 Epoch 60,Training loss 0.20755051261490318
2020-07-19 17:17:06.357585 Epoch 70,Training loss 0.18867057958131384
2020-07-19 17:17:28.880580 Epoch 80,Training loss 0.171582081040759
2020-07-19 17:17:51.508608 Epoch 90,Training loss 0.15744791217860143
2020-07-19 17:18:16.729977 Epoch 100,Training loss 0.14557443900852446


In [44]:
# 计算精度,经过训练，权重还保存着

train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val,batch_size=64,shuffle=True)

def validate(model,train_loader,val_loader):
    for name,loader in[('train',train_loader),('val',val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs,labels in loader:
                outputs = model(imgs)
                _,predicted = torch.max(outputs,dim=1)      # predicted 是索引
                total += labels.shape[0]
                correct +=int((predicted == labels).sum())
              
        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.93
Accuracy val: 0.87


In [49]:
#  保存模型

torch.save(model.state_dict(),data_path + 'bird_vs_airplane.pt')

In [50]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load(data_path + 'bird_vs_airplane.pt'))

<All keys matched successfully>

In [51]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [56]:
import datetime

def training_loop(n_epochs,optimizer,model,loss_fn,train_loader):
    for epoch in range(1,n_epochs+1):
        loss_train = 0.0
        for imgs,labels in train_loader:
            
            imgs = img.to(device = device)          # 输入放到gpu
            labels = labels.to(device=device)
            
            outputs = model(imgs)
            
            loss = loss_fn(outputs,labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch %10 ==0:
            print('{} Epoch {},Training loss {}'.format(
                datetime.datetime.now(),epoch,loss_train/len(train_loader)))
            

In [63]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)

model = Net().to(device=device)                          # 模型放到gpu（包含所有参数）

optimizer = optim.SGD(model.parameters(),lr = 1e-2)

loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader)

2020-07-19 20:05:11.181773 Epoch 1,Training loss 0.6018951303639989
2020-07-19 20:05:18.631052 Epoch 10,Training loss 0.34487766150835975
2020-07-19 20:05:26.798797 Epoch 20,Training loss 0.30025195762230333
2020-07-19 20:05:34.945215 Epoch 30,Training loss 0.2714038026180996
2020-07-19 20:05:43.122595 Epoch 40,Training loss 0.25165232816699207
2020-07-19 20:05:51.554141 Epoch 50,Training loss 0.23327995746568508
2020-07-19 20:06:00.314958 Epoch 60,Training loss 0.2125224553665538
2020-07-19 20:06:08.473854 Epoch 70,Training loss 0.1973991779385099
2020-07-19 20:06:16.520341 Epoch 80,Training loss 0.18535497134468357
2020-07-19 20:06:21.605332 Epoch 90,Training loss 0.16962906590123086
2020-07-19 20:06:24.898841 Epoch 100,Training loss 0.1555779479966042


In [66]:
torch.save(model.state_dict(), data_path + 'birds_vs_airplanes_g.pt')

In [67]:
loaded_model = Net().to(device = device)
loaded_model.load_state_dict(torch.load(data_path +'birds_vs_airplanes_g.pt',map_location = device))

<All keys matched successfully>

In [ ]:
## model design


In [2]:
str = 'v_ApplyEyeMakeup_g01_c01.avi'

x = str.split('.')[1]
x

'avi'